<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>



# <center>Getting Started with Llamatrace

This guide demonstrates how to use Llamatrace, a collaboration between Arize and LlamaIndex to deliver hosted observability and evals with native support for LlamaIndex.

ℹ️ This notebook requires an OpenAI API key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [ ]:
%%bash

pip install -q "arize-phoenix>=4.29.0" openai 'httpx<0.28' openinference-instrumentation-openai
pip install -q gcsfs nest-asyncio "openinference-instrumentation-llama-index>=3.0.0"
pip install -qU llama-index-callbacks-arize-phoenix
pip install -qU llama-index llama-index-llms-openai

In [ ]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    phoenix_api_key = getpass("🔑 Enter your Phoenix API key: ")

os.environ["PHOENIX_API_KEY"] = phoenix_api_key

## Step 2: Setup Tracing
Let's send a trace to Hosted Phoenix! Note the key lines below using `PHOENIX_CLIENT_HEADERS` and `app.phoenix.arize.com/v1/traces`
LlamaIndex has a built-in integration with Phoenix, so we can use their `set_global_handler` method to send traces to Phoenix. You can also use the `register` method from `arize-phoenix-otel` to achieve the same thing.

In [ ]:
# setup Arize Phoenix for logging/observability
import os

from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

from phoenix.otel import register

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

# Configuration is picked up from your environment variables
tracer_provider = register()

# Instrument LlamaIndex. This allows Phoenix to collect traces from LlamaIndex queries.
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)

Setup imports

In [ ]:
import json
import os
from urllib.request import urlopen

import nest_asyncio
import pandas as pd
from gcsfs import GCSFileSystem
from llama_index.core import (
    Settings,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from tqdm import tqdm

import phoenix as px

nest_asyncio.apply()  # needed for concurrent evals in notebook environments
pd.set_option("display.max_colwidth", 1000)

Run LlamaIndex Query on Arize Docs

In [ ]:
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
)

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
index = load_index_from_storage(
    storage_context,
)
query_engine = index.as_query_engine()

queries_url = "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(queries_url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])
queries[:5]

for query in tqdm(queries[:5]):
    query_engine.query(query)

response = query_engine.query("What is Arize and how can it help me as an AI Engineer?")
print(response)

## Step 3: Access your Phoenix instance
You can access your Phoenix instance to power evaluations, experiments, upload datasets, etc by using the px.Client() object.



In [ ]:
px_client = px.Client()
phoenix_df = px_client.get_spans_dataframe()
print(phoenix_df.head())